<img src="https://raw.githubusercontent.com/euroargodev/argopy/master/docs/_static/argopy_logo_long.png" alt="argopy logo" width="200"/>

# Training Camp - Sept 22<sup>th</sup> 2025

***

## Notebook Title : Filtering - QC flags & Data mode

**Author contact : [K. Balem](https://annuaire.ifremer.fr/cv/22144/)**

**Description:**

This notebook describes how to filter & transform your data :
- with QC values
- with Data mode
- with OWC and research mode requirements

This notebook is based on the [Argopy documentation](https://argopy.readthedocs.io/en/v1.3.0/user-guide/working-with-argo-data/data_manipulation.html#) where you can find more details on each function.

*This notebook was developed with Argopy version: 1.3*

***

## 🚩 QC flag filter
You can filter measurements according to QC flag values with the [Dataset.argo.filter_qc()](https://argopy.readthedocs.io/en/v1.3.0/generated/xarray.Dataset.argo.filter_qc.html#xarray.Dataset.argo.filter_qc) method. This filter modifies all variables of the dataset.

In [ ]:
from argopy import DataFetcher

Let's fetch some data in expert mode, so that every QC is present in our dataset

In [ ]:
f = DataFetcher(src='erddap', mode='expert').float(3901971) 
ds = f.data
ds

You can filter your data with all the QC fields available in your dataset : 

In [ ]:
dsq = ds.argo.filter_qc(QC_list=[1,2], QC_fields='all')
dsq

Or you can specify the qualification variable to use when filtering

In [ ]:
dsqs = ds.argo.filter_qc(QC_list=[1,2], QC_fields=['PSAL_QC'])
dsqs

In [ ]:
import matplotlib.pyplot as plt
f,ax = plt.subplots(1,2,figsize=(12,4))
ax[0].plot(ds['PSAL'],ds['TEMP'],'k.',markersize=0.2)
ax[0].plot(dsq['PSAL'],dsq['TEMP'],'g.',markersize=0.2)
ax[1].plot(ds['PSAL'],ds['TEMP'],'k.',markersize=0.2)
ax[1].plot(dsqs['PSAL'],dsqs['TEMP'],'r.',markersize=0.2)

#### ✏️ EXERCICE
Fetch the data from your favorite float in expert mode, and create a dataset with only good data (QC=[1,2]).

## 📉 Data mode filter 
You can filter variables according to their data mode with the method [Dataset.argo.datamode.filter()](https://argopy.readthedocs.io/en/v1.3.0/generated/xarray.Dataset.argo.datamode.filter.html#xarray.Dataset.argo.datamode.filter)  
Filter the dataset to keep points where all or some of the parameters are in any of the data mode specified.

In [ ]:
dsdm = ds.argo.datamode.filter(dm=['D'], params='all')
dsdm

#### ✏️ EXERCICE
Fetch the data from float 1902574 in expert mode and create a real-time dataset and and a delayed-mode one

## ♻️ Data mode based transform
Note that you can also use the datamode variable to transform your dataset. The [Dataset.argo.datamode.merge()](https://argopy.readthedocs.io/en/v1.3.0/generated/xarray.Dataset.argo.datamode.merge.html#xarray.Dataset.argo.datamode.merge) method will copy any ADJUSTED values onto Variable for points where param data mode is 'A' or 'D'. After values have been copied, all ADJUSTED variables are dropped to avoid confusion.

In [ ]:
dsdm_merged = ds.argo.datamode.merge()
dsdm_merged

Note that all the ADJUSTED variables have been dropped, to only keep simple and clearer variable names

## 🌊 OWC variables filter 
You can filter variables according to OWC salinity calibration software requirements with the method [Dataset.argo.filter_scalib_pres()](https://argopy.readthedocs.io/en/v1.3.0/generated/xarray.Dataset.argo.filter_scalib_pres.html#xarray.Dataset.argo.filter_scalib_pres).  
This filter modifies pressure, temperature and salinity related variables of the dataset.

By default, this filter will return a dataset with raw PRES, PSAL and TEMP; and if PRES is adjusted, PRES variable will be replaced by PRES_ADJUSTED.With option force=’raw’, you can force the filter to return a dataset with raw PRES, PSAL and TEMP whether PRES is adjusted or not. With option force=’adjusted’, you can force the filter to return a dataset where PRES/PSAL and TEMP replaced with adjusted variables: PRES_ADJUSTED, PSAL_ADJUSTED, TEMP_ADJUSTED. Since ADJUSTED variables are not required anymore after the filter, all ADJUSTED variables are dropped in order to avoid confusion wrt variable content.

In [ ]:
ds_owc = ds.argo.filter_scalib_pres(force='default')
ds_owc

## 🔍 Research mode filter 
Filter dataset for [research user mode](https://argopy.readthedocs.io/en/v1.3.0/user-guide/fetching-argo-data/user_mode.html#mode-research-research-mode). This filter depends on the dataset:
- For the ``phy`` dataset (core/deep missions): select delayed mode data with QC=1 and with pressure errors smaller than 20db
- For the ``bgc`` dataset: do nothing, filtering for the ``research`` user mode is implemented in the fetcher facade

In [ ]:
f = DataFetcher(src='erddap', mode='expert').float(3901996) 
ds = f.data
ds

In [ ]:
ds_rm = ds.argo.filter_researchmode()
ds_rm

#### ✏️ EXERCICE
Fetch the data from float 4903173 in "expert" mode and filter the dataset with research mode filter. Compare the result with data fetched in "research" mode directly. 

### 
***
Useful argopy commands:
```python
argopy.reset_options()
argopy.show_options()
argopy.status()
argopy.clear_cache()
argopy.show_versions()
```
***
![logo](https://raw.githubusercontent.com/euroargodev/argopy-training/refs/heads/main/for_nb_producers/template_argopy_training_EAONE.png)